In [155]:
import os
import pandas as pd
import requests

In [156]:
folder='all'
columns=['handle','url','provider-homepage','template-type']
all_items=[]
for file in os.listdir(folder)[0:1000]:
    if file.endswith('.rdf'):
        with open(folder+'/'+file,'r') as rdf_file:
            rdf_string=rdf_file.read()
            rdf_list= rdf_string.split('\n')
            rdf_item_dict={}
            for rdf_item in rdf_list:
                if ':' not in rdf_item: 
                    continue
                key=rdf_item.split(':')[0]
                if 1==2 or key.lower() in columns:
                    value=rdf_item.replace(key+':','')
                    rdf_item_dict[key.strip().lower()]=value.strip()
            all_items.append(rdf_item_dict)

In [123]:
df=pd.DataFrame(all_items)

In [165]:
html_urls=df[df['template-type']=='ReDIF-Archive 1.0'][df['url'].str[0:3]=='htt'].url.values

/Users/mahdimoqri/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [169]:
for url in html_urls[0:1]:
    r = requests.get(url)
    print r.status_code

ConnectionError: HTTPSConnectionPool(host='druid.dkRePEc', port=443): Max retries exceeded with url: /aal/ (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x1121d7810>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))

In [170]:
url

'http://www.druid.dk/RePEc/aal/'

In [171]:
r = requests.get(url)

ConnectionError: HTTPSConnectionPool(host='druid.dkRePEc', port=443): Max retries exceeded with url: /aal/ (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x1121d7990>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known',))